# Predicting Price with Location (City) and Land Use

## Prepare

### Import

In [1]:
import pandas as pd
import seaborn as sns
from category_encoders import OneHotEncoder
from ipywidgets import Dropdown, interact
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline

df = pd.read_csv ("C:/Users/Muibat Odunukan/Documents/personal/Portfolio Projects/Dataset/Cleaned_Nashville.csv")
df.head()

,UniqueID,ParcelID,LandUse,SalePrice,LegalReference,SoldAsVacant,OwnerName,Acreage,LandValue,BuildingValue,...,YearBuilt,Bedrooms,FullBath,HalfBath,SaleDateConverted,PropertySplitAddress,PropertySplitCity,OwnerSplitAddress,OwnerSplitCity,OwnerSplitState
0,2045,007 00 0 125.00,SINGLE FAMILY,240000,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE",2.3,50000.0,168200.0,...,1986.0,3.0,3.0,0.0,4/9/2013,1808 FOX CHASE DR,GOODLETTSVILLE,1808 FOX CHASE DR,GOODLETTSVILLE,TN
1,16918,007 00 0 130.00,SINGLE FAMILY,366000,20140619-0053768,No,"BONER, CHARLES & LESLIE",3.5,50000.0,264100.0,...,1998.0,3.0,3.0,2.0,6/10/2014,1832 FOX CHASE DR,GOODLETTSVILLE,1832 FOX CHASE DR,GOODLETTSVILLE,TN
2,54582,007 00 0 138.00,SINGLE FAMILY,435000,20160927-0101718,No,"WILSON, JAMES E. & JOANNE",2.9,50000.0,216200.0,...,1987.0,4.0,3.0,0.0,9/26/2016,1864 FOX CHASE DR,GOODLETTSVILLE,1864 FOX CHASE DR,GOODLETTSVILLE,TN
3,43070,007 00 0 143.00,SINGLE FAMILY,255000,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.",2.6,50000.0,147300.0,...,1985.0,3.0,3.0,0.0,1/29/2016,1853 FOX CHASE DR,GOODLETTSVILLE,1853 FOX CHASE DR,GOODLETTSVILLE,TN
4,22714,007 00 0 149.00,SINGLE FAMILY,278000,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.",2.0,50000.0,152300.0,...,1984.0,4.0,3.0,0.0,10/10/2014,1829 FOX CHASE DR,GOODLETTSVILLE,1829 FOX CHASE DR,GOODLETTSVILLE,TN


### Explore

In [2]:
# Subsetting all colums except "UniqueID". It is not useful 

df = df.iloc[:,1:]
df.head()

,ParcelID,LandUse,SalePrice,LegalReference,SoldAsVacant,OwnerName,Acreage,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath,SaleDateConverted,PropertySplitAddress,PropertySplitCity,OwnerSplitAddress,OwnerSplitCity,OwnerSplitState
0,007 00 0 125.00,SINGLE FAMILY,240000,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE",2.3,50000.0,168200.0,235700.0,1986.0,3.0,3.0,0.0,4/9/2013,1808 FOX CHASE DR,GOODLETTSVILLE,1808 FOX CHASE DR,GOODLETTSVILLE,TN
1,007 00 0 130.00,SINGLE FAMILY,366000,20140619-0053768,No,"BONER, CHARLES & LESLIE",3.5,50000.0,264100.0,319000.0,1998.0,3.0,3.0,2.0,6/10/2014,1832 FOX CHASE DR,GOODLETTSVILLE,1832 FOX CHASE DR,GOODLETTSVILLE,TN
2,007 00 0 138.00,SINGLE FAMILY,435000,20160927-0101718,No,"WILSON, JAMES E. & JOANNE",2.9,50000.0,216200.0,298000.0,1987.0,4.0,3.0,0.0,9/26/2016,1864 FOX CHASE DR,GOODLETTSVILLE,1864 FOX CHASE DR,GOODLETTSVILLE,TN
3,007 00 0 143.00,SINGLE FAMILY,255000,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.",2.6,50000.0,147300.0,197300.0,1985.0,3.0,3.0,0.0,1/29/2016,1853 FOX CHASE DR,GOODLETTSVILLE,1853 FOX CHASE DR,GOODLETTSVILLE,TN
4,007 00 0 149.00,SINGLE FAMILY,278000,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.",2.0,50000.0,152300.0,202300.0,1984.0,4.0,3.0,0.0,10/10/2014,1829 FOX CHASE DR,GOODLETTSVILLE,1829 FOX CHASE DR,GOODLETTSVILLE,TN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56373 entries, 0 to 56372
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ParcelID              56373 non-null  object 
 1   LandUse               56373 non-null  object 
 2   SalePrice             56373 non-null  int64  
 3   LegalReference        56373 non-null  object 
 4   SoldAsVacant          56373 non-null  object 
 5   OwnerName             25215 non-null  object 
 6   Acreage               25969 non-null  float64
 7   LandValue             25969 non-null  float64
 8   BuildingValue         25969 non-null  float64
 9   TotalValue            25969 non-null  float64
 10  YearBuilt             24118 non-null  float64
 11  Bedrooms              24112 non-null  float64
 12  FullBath              24230 non-null  float64
 13  HalfBath              24099 non-null  float64
 14  SaleDateConverted     56373 non-null  object 
 15  PropertySplitAddres

In [4]:
df.select_dtypes("object").nunique()

ParcelID                48559
LandUse                    39
LegalReference          52761
SoldAsVacant                2
OwnerName               19713
SaleDateConverted        1119
PropertySplitAddress    45061
PropertySplitCity          14
OwnerSplitAddress       22326
OwnerSplitCity             12
OwnerSplitState             1
dtype: int64

In [5]:
#Drop columns with low- and high- cardinality categorical features
df.drop(columns=[
    "SoldAsVacant", 
    "OwnerSplitState", 
    "ParcelID", 
    "LegalReference", 
    "OwnerName", 
    "SaleDateConverted", 
    "PropertySplitAddress", 
    "OwnerSplitAddress"''    
], inplace=True)

In [6]:
df.isnull().sum() / len(df) * 100

LandUse               0.000000
SalePrice             0.000000
Acreage              53.933621
LandValue            53.933621
BuildingValue        53.933621
TotalValue           53.933621
YearBuilt            57.217107
Bedrooms             57.227751
FullBath             57.018431
HalfBath             57.250812
PropertySplitCity     0.000000
OwnerSplitCity       53.933621
dtype: float64

In [7]:
#Drop columns with more than 50% null values

df.drop(columns=[
    "Acreage", 
    "LandValue", 
    "BuildingValue", 
    "TotalValue", 
    "YearBuilt", 
    "Bedrooms", 
    "FullBath", 
    "HalfBath",
    "OwnerSplitCity"
], inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56373 entries, 0 to 56372
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   LandUse            56373 non-null  object
 1   SalePrice          56373 non-null  int64 
 2   PropertySplitCity  56373 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


   ### 1.3 Split

In [9]:
# Seperate the Feature Matrix (X) from Target vector (y)
target = "SalePrice"
features = ["LandUse", "PropertySplitCity"]
X = df[features]
y = df[target]

In [10]:
#Split into 70-30
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.30, random_state=42)

## Build Model

### Baseline

In [11]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] *len(y_train)

print("Mean housing Price:", round(y_mean, 2))
print("Basline_MAE:", mean_absolute_error(y_train, y_pred_baseline))

Mean housing Price: 332382.34
Basline_MAE: 239043.0101595623


### Iterate 

In [12]:
#create pipeline that contains a OneHotEncoder and Ridge predictor.

model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    Ridge()
)                          
model.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['LandUse', 'PropertySplitCity'],
                               use_cat_names=True)),
                ('ridge', Ridge())])

### Evaluate

In [13]:
y_pred_training = model.predict(X_train)
print("Training MAE:", mean_absolute_error(y_train, y_pred_training))

Training MAE: 237155.97824220566


In [14]:
y_pred_test = pd.Series(model.predict(X_test))
print("Training MAE:", mean_absolute_error(y_test, y_pred_test))

Training MAE: 221917.14186512155


## Communicate Result

In [15]:
# Create a prediction function to deploy model 

def make_prediction(LandUse, City):
    data ={
        "LandUse" : LandUse,
        "PropertySplitCity" : City
    }
    df = pd.DataFrame(data, index=[0])
    prediction = model.predict(df).round(2)[0]
    return f"Predicted apartment price: ${prediction}"

In [16]:
make_prediction("SINGLE FAMILY", "JOELTON")

'Predicted apartment price: $197811.04'

In [17]:
# Create an interacttive Dashboard to deploy trained model

interact(
    make_prediction,
    LandUse=Dropdown(options=sorted(X_train["LandUse"].unique())),
    City=Dropdown(options=sorted(X_train["PropertySplitCity"].unique()))
);

interactive(children=(Dropdown(description='LandUse', options=('APARTMENT: LOW RISE (BUILT SINCE 1960)', 'CHUR…